In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# os.environ['JAX_PLATFORM_NAME'] = "cpu"
# os.environ['JAX_PLATFORMS'] = "cpu"

In [ ]:
import jax.numpy as jnp
import jax

import numpy as onp

import matplotlib.pyplot as plt
import random

from simulation.simulate_full import run_entire_simulation

from analyzers import defaultvalues as dv, database, loss as loss_anaylzer, gradients as grad_analyzer

database.set_filename("../data/grad_analyzer/nq_scan.npz")

In [ ]:
N_GRADIENTS = 4
nq_values = onp.array([50, 100, 150, 200, 250])
print(nq_values)

In [4]:
def print_info(grads):
    amean, astd = grad_analyzer.analyze_gradients_absolute(grads)
    mmean, mstd = grad_analyzer.analyze_gradients_magnitudal(grads)

    print(f"Absolute mean: {amean}, Absolute std: {astd}")
    print(f"Magnitudal mean: {mmean}, Magnitudal std: {mstd}")

In [ ]:
for nq in nq_values:
    existing_keys = database.get_existing_keys()
    if nq in existing_keys:
        print(f"Skipping {nq}, was already computed")
        continue

    grads = []
    print(f"==== for {nq} Q-values ====")
    for i in range(1, N_GRADIENTS + 1):
        key = random.randrange(0, 20000)
        def simulation_wrapper(LJ_SIGMA_OO: float) -> float:
            prediction = run_entire_simulation(
                LJ_SIGMA_OO, 
                dv.N_STEPS, 
                dv.N_MOLECULES_PER_AXIS, 
                dv.N_SNAPSHOTS, 
                nq, 
                key)
            reference = run_entire_simulation(
                dv.LJ_SIGMA_OO, 
                dv.N_STEPS, 
                dv.N_MOLECULES_PER_AXIS, 
                dv.N_SNAPSHOTS, 
                nq, 
                key)
            return loss_anaylzer.L1_loss(prediction, reference)
        grad_fn = jax.grad(simulation_wrapper)
        grad = grad_fn(3.1)
        print(grad)
        grads.append(grad)

    print_info(grads)
    database.save_intermediate_result(nq, grads)

In [ ]:
keys, values = database.load_result()
for key, grads in zip(keys, values):
    print(f"==== for {nq} Q-values ====")
    print_info(grads)